In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### Load the trained model. scaler pickle , onehot_encoder

model = load_model("model.h5")


## load encoder and scalers

with open('lable_encoder_gender.pkl','rb') as file:
    lable_encoder_gender = pickle.load(file)


with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)


with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [4]:
## Example input data

input_data={ 'CreditScore': 600,
            'Geography': 'France',
            'Gender': 'Male',
            'Age': 40,
            'Tenure': 3,
            'Balance': 60000,
            'NumOfProducts': 2,
            'HasCrCard': 1,
            'IsActiveMember': 1,
            'EstimatedSalary': 50000
}


In [5]:
# One- hot encode "Geography"

geo_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

C:\Users\Prathmesh\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [9]:
## Combine one-hot coded columns with input data
input_data = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
input_data["Gender"] = lable_encoder_gender.transform(input_data["Gender"])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [17]:
input_data = input_data.drop(columns=["Geography"],axis=1)

input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
input_scaled = scaler.transform(input_data)

input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
prediction = model.predict(input_scaled)

prediction

1/1 [==============================] - 0s 121ms/step


array([[0.05140369]], dtype=float32)

In [22]:
prediction_probability = prediction[0][0]
prediction_probability

0.051403686

In [23]:
if prediction_probability > 0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
